In [ ]:
import os
import sys
import gzip

import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

import tqdm
from joblib import Parallel, delayed
from multiprocessing import Manager

sys.path.append('../..')
from slp.slp_package.slp_functions import create_merged_game_data_df
from slp.slp_package.input_dataset import InputDataSet

In [ ]:
source_data = ['ranked']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}
    

In [ ]:
df = pd.read_parquet('/workspace/melee_project_data/data/mango_no_teams_2_players_df.parquet')

In [ ]:
print(df.columns)
df.head()

In [ ]:
path = '/workspace/melee_project_data/input_np/ranked/FOX/441acea3-4e1b-463a-8063-95479aa39cf0.npy.gz'

with gzip.open(path, 'rb') as f:
    inputs_array = np.load(f)

In [ ]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

In [ ]:
dataset.dataset.value_counts()
dataset.dataset.head()

In [ ]:
dataset.number_of_segments_per_game(10, 10000)

In [ ]:
X_train, X_test, y_train, y_test  = dataset.train_test_split_numpy(test_ratio = .30,  val = False)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
from sktime.classification.deep_learning.cnn import CNNClassifier

batch_size = 32
epochs = 10
model = CNNClassifier(n_epochs=epochs, batch_size=batch_size, loss = 'categorical_crossentropy')

model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

